**NOTE** Requires to ffmpeg codecs installed and a new matplotlib with viridis colormap

In [1]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.animation as animation
import sys
sys.path.append('/home/3y9/Mantid/Build/bin')
from mantid.simpleapi import CreateMDHistoWorkspace, LoadMD

class MakeMovie(object):
    def __init__(self,MDWS,axis,filename,**kwargs):
        self.signal=MDWS.getSignalArray()*1.
        self.dims=[MDWS.getDimension(i) for i in (0,1,2)]
        self.perp_axis=self.dims.pop(axis)
        self.perp_axis_index=axis
        self.fig,self.ax=plt.subplots()
        self.slicer=[slice(None),]*3
        self.slicer[axis]=0
        self.pcm=self.Plot2DMD(self.dims[0],self.dims[1],self.signal[self.slicer],**kwargs)
        self.title=self.ax.set_title("{0}={1:3.3f}".format(self.perp_axis.getName(),
                                                           0.5*(self.perp_axis.getX(0)+self.perp_axis.getX(1))))
        self.ax.set_xlim(self.dims[0].getMinimum(),self.dims[0].getMaximum())
        self.ax.set_ylim(self.dims[1].getMinimum(),self.dims[1].getMaximum())
        self.fig.colorbar(self.pcm,ax=self.ax)
        self.ani = animation.FuncAnimation(self.fig, self.update_fig, frames=self.perp_axis.getNBins(), interval=50, blit=True)
        self.ani.save(filename, writer = "ffmpeg", extra_args=['-vcodec', 'libx264'])
        plt.close(self.fig)

    def update_fig(self,index):
        #extract the new slice
        self.slicer=[slice(0,-1),]*3
        self.slicer[self.perp_axis_index]=index
        data=np.array(self.signal[self.slicer].ravel())
        data=np.ma.masked_where(np.isnan(data),data)
        #set data for the colormesh
        self.pcm.set_array(data)
        #set title
        self.title.set_text("{0}={1:3.3f}".format(self.perp_axis.getName(),
                                                  0.5*(self.perp_axis.getX(index)+self.perp_axis.getX(index+1))))
        return self.pcm,self.title 

    def dim2array(self,d):
        """
        Create a numpy array containing bin centers along the dimension d
        input: d - IMDDimension
        return: numpy array, from min+st/2 to max-st/2 with step st  
        """
        dmin=d.getMinimum()
        dmax=d.getMaximum()
        dstep=d.getX(1)-d.getX(0)
        return np.arange(dmin+dstep/2,dmax,dstep)

    def Plot2DMD(self,dimx,dimy,intensity,**kwargs):
        """
        Plot a 2D slice 
        """
        x=self.dim2array(dimx)
        y=self.dim2array(dimy)
        intensity=np.ma.masked_where(np.isnan(intensity),intensity)
        XX,YY=np.meshgrid(x,y,indexing='ij')
        pcm=self.ax.pcolormesh(XX,YY,intensity,**kwargs)
        self.ax.set_xlabel(dimx.getName())
        self.ax.set_ylabel(dimy.getName())
        return pcm

In [2]:
def ll(x,xim1,Qc):
    """
    Lattice lorentzian = sum of lorentzians equally spaced on integers
    """
    return np.sinh(xim1)/(np.cosh(xim1)-np.cos(2*np.pi*(x+Qc)))

def simulate_data():
    xx,yy,zz=np.mgrid[-8:8:200j,-8:8:200j,-8:8:200j]

    #"magnetic scattering"
    xiMm1=1.
    intensityM=ll(xx,xiMm1,0.5)*ll(yy,xiMm1,0.5)*ll(zz,xiMm1,0.5)
    intensityM=(intensityM-intensityM.min())*100.
    intensityM*=np.exp(-(xx**2+yy**2+zz**2)/8.)
    #"Bragg scattering"
    xiBm1=5.e-2
    intensityB=ll(xx,xiBm1,0.)*ll(yy,xiBm1,0.)*ll(zz,xiBm1,0.)

    intensity=intensityB+intensityM
    intensity=intensity.flatten()
    ws=CreateMDHistoWorkspace(Dimensionality=3,
                              Extents='-5,5,-5,5,-5,5',
                              SignalInput=intensity,
                              ErrorInput=intensity,
                              NumberOfBins='200,200,200',
                              Names='H,K,L',
                              Units='MomentumTransfer,MomentumTransfer,MomentumTransfer')
    return ws

In [3]:
#ws= LoadMD(Filename='/SNS/TOPAZ/IPTS-17133/shared/Sr_100K/Visualization/3D_vol_full/T90K_combined_symm-oP.nxs', LoadHistory=False)
ws=simulate_data()
mov=MakeMovie(ws,0,"../img/KLslices.mp4",norm=LogNorm(vmin=1e-1,vmax=1e2),cmap='viridis')

In [4]:
%%HTML
<video width="800" height="600" controls>
  <source src="../img/KLslices.mp4" type="video/mp4">
</video>